In [36]:
import numpy as np 

# importing necessary libraries to use throughout

import tensorflow
import random
import sys
import os
import h5py
import nltk
import string
import collections
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import 	WordNetLemmatizer

#importing layers,models and optimizers along with callbacks from Keras

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from nltk.tokenize import word_tokenize


# loading .txt file 
path = 'nopunc2.txt'
text = open(path).read().lower() # reading text
print('Corpus length:', len(text)) #printing length of text
#print(text)

Corpus length: 157993


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/marlontabone/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marlontabone/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [37]:
def clean_doc(doc):

     clean_words= []

     words = word_tokenize(text)
     for word in words:
           word = word.strip(string.punctuation)
           if len(word)>=1 and word.isdigit()==False:
                      word = word.lower()
                      clean_words.append(word)
     return clean_words

token=clean_doc(text)

wd = WordNetLemmatizer()
newDict = []

for w in token:
    newDict.append(wd.lemmatize(w))
print(newDict) 

['former', 'opposition', 'leader', 'simon', 'busuttil', 'testified', 'in', 'court', 'this', 'morning', 'a', 'did', 'the', 'prime', 'ministeräôs', 'more', 'former', 'opposition', 'leader', 'simon', 'busuttil', 'testified', 'in', 'court', 'this', 'morning', 'a', 'did', 'the', 'prime', 'ministeräôs', 'more', 'whenever', 'the', 'leader', 'of', 'the', 'labour', 'party', 'is', 'asked', 'question', 'about', 'the', 'more', 'embarrassing', 'aspect', 'of', 'his', 'past', 'he', 'say', 'heäôll', 'leave', 'it', 'to', 'the', 'historian', 'to', 'decide', 'because', 'a', 'far', 'a', 'heäôs', 'concerned', 'itäôs', 'all', 'water', 'under', 'the', 'bridge', 'and', 'he', 'ha', 'no', 'regret', 'and', 'this', 'when', 'nobody', 'ha', 'bothered', 'to', 'ask', 'him', 'yet', 'what', 'he', 'thought', 'of', 'the', 'labour', 'governmentäôs', 'corruption', 'and', 'terrible', 'moral', 'and', 'physical', 'violence', 'in', 'the', 'day', 'when', 'he', 'wa', 'president', 'of', 'the', 'labour', 'party', 'when', 'sant', '

In [69]:
sequence_len=30
seq=[]


#Was meant to use newDict() but instead used token 
for i in range(0,len(token)-sequence_len):
    seq.append(token[i:i+sequence_len])
    
tokenizer = Tokenizer()
tokenizer.fit_on_texts(seq)
sequence = tokenizer.texts_to_sequences(seq)
print(sequence[2])

vocab_size = len(tokenizer.word_index)
print(vocab_size)

import numpy as np
arr=np.array(sequence)

[100, 1140, 1138, 1137, 7, 729, 15, 194, 21, 132, 1, 54, 728, 82, 1141, 561, 100, 1140, 1138, 1137, 7, 729, 15, 194, 21, 132, 1, 54, 728, 82]
2072


In [56]:
X, Y = arr[:,:-1], arr[:,-1]

Y=to_categorical(Y,num_classes=vocab_size+1)

print(Y.shape)

seq_length = X.shape[1]
print(seq_length)

(27964, 2073)
29


In [67]:
from tensorflow.keras.layers import LSTM, Input, Embedding, Dense
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint 


# using a function to build model architecture in case we may use it again 

model = models.Sequential()
model.add(Embedding(vocab_size+1, 150,input_length=seq_length))
model.add(LSTM(528,return_sequences = True))
model.add(LSTM(528))
model.add(layers.Dense(528, activation = 'relu'))
model.add(layers.Dense(vocab_size+1, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 29, 150)           310950    
_________________________________________________________________
lstm_8 (LSTM)                (None, 29, 528)           1434048   
_________________________________________________________________
lstm_9 (LSTM)                (None, 528)               2232384   
_________________________________________________________________
dense_8 (Dense)              (None, 528)               279312    
_________________________________________________________________
dense_9 (Dense)              (None, 2073)              1096617   
Total params: 5,353,311
Trainable params: 5,353,311
Non-trainable params: 0
_________________________________________________________________


In [68]:
model.fit(X, Y, batch_size=80, epochs=100)

Train on 27964 samples
Epoch 1/100
27964/27964 [==============================] - 254s 9ms/sample - loss: 6.3649 - accuracy: 0.0527
Epoch 2/100
27964/27964 [==============================] - 266s 9ms/sample - loss: 5.8722 - accuracy: 0.0616
Epoch 3/100
27964/27964 [==============================] - 258s 9ms/sample - loss: 5.5604 - accuracy: 0.0762
Epoch 4/100
27964/27964 [==============================] - 264s 9ms/sample - loss: 5.2214 - accuracy: 0.0845
Epoch 5/100
27964/27964 [==============================] - 269s 10ms/sample - loss: 4.9223 - accuracy: 0.0926
Epoch 6/100
27964/27964 [==============================] - 271s 10ms/sample - loss: 4.7112 - accuracy: 0.1044
Epoch 7/100
27964/27964 [==============================] - 272s 10ms/sample - loss: 4.3701 - accuracy: 0.1224
Epoch 8/100
27964/27964 [==============================] - 271s 10ms/sample - loss: 3.9939 - accuracy: 0.1600
Epoch 9/100
27964/27964 [==============================] - 270s 10ms/sample - loss: 3.4556 - accuracy

In [93]:
# Length of extracted word sequences
maxlen = 30

# We sample a new sequence every `step` words
step = 1

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up words)
next_words = []

for i in range(0, len(token) - maxlen, step):
    sentences.append(token[i: i + maxlen])
    next_words.append(token[i + maxlen])


In [122]:
import random as rand

tokenised_text = sentences[rand.randrange(len(sentences))]

from nltk.tokenize.treebank import TreebankWordDetokenizer
seed_text = TreebankWordDetokenizer().detokenize(tokenised_text)

#print(seed_text)

def generate_seq(model,tokenizer,seq_length,seed_text,n_words):

     result = []
     in_text =seed_text
     for i in range(n_words):
          encoded = tokenizer.texts_to_sequences([in_text])[0]
          encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
          yhat = model.predict_classes(encoded)
          out_word = ''
          for word,index in tokenizer.word_index.items():
               if index==yhat:
                  out_word = word
                  break
          in_text+=' ' + out_word
          result.append(word)
     return ' '.join(result)

generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)

print(generated)

home ground and with jason playing robin to physical batman and the malta heavies who going to past for labour because he thought when the truth is a real summing move one up pullicino orlando is of to its answer what pullicino orlando in a man and patience of management


In [123]:
model.save("modelTEST.h5")
print("Saved model to disk")

Saved model to disk
